In [1]:
import itertools
import operator
import numpy as np
import nltk
import sys
import os
import time
from datetime import datetime
nltk.download('punkt')

vocabulary_size=20000
unknown_token = "UNKNOWN"
sentence_start_token = "START"
sentence_end_token = "END"


#preprocess data
with open("/Users/taohuadao/Downloads/vac.txt",'rb') as f:
    reader=f.readlines()
    sentences = itertools.chain(*[nltk.sent_tokenize(x.decode('utf-8').lower()) for x in reader])
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    print(sentences[0:7])
# print "Parsed %d sentences." % (len(sentences))


tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
print(tokenized_sentences[0])
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print(word_freq)
print "Found %d unique words tokens." % len(word_freq.items())
vocab = word_freq.most_common(vocabulary_size-1)
print([vocab[0:3]])

index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])
print(X_train[0:10],y_train[0:10])

def softmax(x):
    """
    对输入x的每一行计算softmax。

    该函数对于输入是向量（将向量视为单独的行）或者矩阵（M x N）均适用。
    
    代码利用softmax函数的性质: softmax(x) = softmax(x + c)

    参数:
    x -- 一个N维向量，或者M x N维numpy矩阵.

    返回值:
    x -- 在函数内部处理后的x
    """
    orig_shape = x.shape

    # 根据输入类型是矩阵还是向量分别计算softmax
    if len(x.shape) > 1:
        # 矩阵
        tmp = np.max(x,axis=1) # 得到每行的最大值，用于缩放每行的元素，避免溢出
        x -= tmp.reshape((x.shape[0],1)) # 利用性质缩放元素
        x = np.exp(x) # 计算所有值的指数
        tmp = np.sum(x, axis = 1) # 每行求和        
        x /= tmp.reshape((x.shape[0], 1)) # 求softmax
    else:
        # 向量
        tmp = np.max(x) # 得到最大值
        x -= tmp # 利用最大值缩放数据
        x = np.exp(x) # 对所有元素求指数        
        tmp = np.sum(x) # 求元素和
        x /= tmp # 求somftmax
    return x
#Model
class RNN:
    def __init__(self, word_dim, hidden_dim = 100, bptt_truncate=4):
        self.word_dim = word_dim #单词维度
        self.hidden_dim = hidden_dim # 隐藏层数量
        self.bptt_truncate = bptt_truncate
        # 输入权重矩阵 H*K 
        self.U = np.random.uniform(-np.sqrt(1.0/word_dim), np.sqrt(1.0/word_dim), (hidden_dim, word_dim))
        # 隐藏层权重矩阵 H*H
        self.V = np.random.uniform(-np.sqrt(1.0/hidden_dim), np.sqrt(1.0/hidden_dim), (word_dim, hidden_dim))
        # 输出层权重矩阵 K*H
        self.W = np.random.uniform(-np.sqrt(1.0/hidden_dim), np.sqrt(1.0/hidden_dim), (hidden_dim, hidden_dim))
    def forward_propagation(self, x):
        T=len(x)
        # 隐藏层各时刻输出
        s = np.zeros((T+1, self.hidden_dim))
        # 各时刻实际输出
        o = np.zeros((T, self.word_dim))
        for t in np.arange(T):
            s[t] = np.tanh(self.W.dot(s[t-1]) + self.U[:,x[t]])
            o[t] = softmax(self.V.dot(s[t]))
        return [o,s]
    def predict(self, x):
        '''
        对x进行前向传播后选择得分最高的index
        Args:
            x: 输入句子序列，一个句子
        Return:index
        '''
        o,s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    def calc_total_loss(self, X_train, Y_train):
        '''
        计算交叉熵损失
        Args:
            X_train: 训练集
            Y_train: 训练集标签
        Return:
            E:训练集上的交叉熵
        '''
        E = 0
        for x,y in zip(X_train, Y_train):
            # 对每个样例
            o, s = self.forward_propagation(x)
            # 取出对每个word后的正确word的估计
            correct_word_predictions = o[np.arange(len(y)),y]
            # 计算熵
            E += -1 * np.sum(np.log(correct_word_predictions))
        return E

    def calc_avg_loss(self, X_train, Y_train):
        '''
        计算每个单词上的平均交叉熵
        '''
        num_total_words = np.sum((len(y_i) for y_i in Y_train))
        return self.calc_total_loss(X_train, Y_train) / num_total_words
    
    def bptt(self,x,y):
        '''
        反向传播
        Args:
            x: 输入句子序列，一个句子
            y：输出句子序列
        Return:
            dLdU: U的梯度
            dLdV: V的梯度
            dLdW: W的梯度
        '''
        T = len(y)
        #进行前向传播
        o,s = self.forward_propagation(x)
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)

        # y^t-yt, 用来表示预测值与真实值之间的差
        delta_o = o
        delta_o[np.arange(len(y)),y] -=1.

        # For each output backwards
        for t in np.arange(T)[::-1]:
            # 计算V的梯度
            dLdV += np.outer(delta_o[t], s[t].T)
            # 初始化的delta计算
            delta_t = self.V.T.dot(delta_o[t]) * (1-s[t] ** 2)
            # BPTT
            for bptt_step in np.arange(max(0,t-self.bptt_truncate),t+1)[::-1]:
                # 累加W的梯度
                dLdW += np.outer(delta_t, s[bptt_step-1])
                # 计算U的梯度
                dLdU[:,x[bptt_step]] += delta_t

                #下一步更新delta
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)

        return [dLdU, dLdV, dLdW]
    
    def sgd(self,x,y,learning_rate):
        '''
        随即梯度下降
        Args:
            x: 输入句子序列，一个句子
            y：输出句子序列
            learning_rate: 学习率
        '''
        dLdU, dLdV, dLdW = self.bptt(x, y)
        self.W -= learning_rate * dLdW
        self.V -= learning_rate * dLdV
        self.U -= learning_rate * dLdU

    def train_with_sgd(self, X_train, Y_train, nepoch = 100, learning_rate = 0.005, evaluate_loss_after = 5):
        '''
        用随机梯度下降训练模型
        Args:
            X_train: 训练集
            Y_train: 训练集标签
            nepoch: 轮询次数
            learning_rate: 学习率

        '''
        losses = []
        num_examples_seen = 0
        for epoch in range(nepoch):
            # optionally evaluate the loss
            if(epoch % evaluate_loss_after == 0):
                loss = self.calc_avg_loss(X_train, Y_train)
                losses.append((num_examples_seen, loss))
                time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)

                #如果loss增加了，那么就减小学习率
                if(len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                    learning_rate = learning_rate * 0.5
                    print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()

            #对每一个训练样本，进行SGD
            for i in range(len(Y_train)):
                self.sgd(X_train[i], Y_train[i], learning_rate)
                num_examples_seen += 1

[nltk_data] Downloading package punkt to /Users/taohuadao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[u'START there were two guys at a bar. END', u'START one of them was rich and the other was poor. END', u'START they both start talking and they find out their anniversary is on the same day, which is tomorrow. END', u'START poor guy- "what did you get your wife?" END', u'START rich guy- "i got her a diamond ring and a mercedes benz." END', u'START poor guy- "why did you give her those??" END', u'START rich guy- "because if she doesn\'t like the ring she can run the car off a cliff and go screw herself. END']
[u'START', u'there', u'were', u'two', u'guys', u'at', u'a', u'bar', u'.', u'END']
<FreqDist with 29641 samples and 750963 outcomes>
Found 29641 unique words tokens.
[[(u'START', 49526), (u'END', 49526), (u'the', 30824)]]
(array([list([0, 58, 78, 95, 432, 40, 5, 237, 3]),
       list([0, 42, 12, 80, 22, 844, 8, 2, 112, 22, 632, 3]),
       list([0, 38, 281, 379,

In [2]:
if __name__ == '__main__':
    model = RNN(10000,100,4)
    model.train_with_sgd(X_train, y_train)

2019-01-25 07:41:51: Loss after num_examples_seen=0 epoch=0: 9.903466
2019-01-25 07:53:40: Loss after num_examples_seen=2500 epoch=5: 6.265628
2019-01-25 08:04:56: Loss after num_examples_seen=5000 epoch=10: 5.735370
2019-01-25 08:16:05: Loss after num_examples_seen=7500 epoch=15: 5.540550
2019-01-25 08:27:45: Loss after num_examples_seen=10000 epoch=20: 5.408340
2019-01-25 08:39:03: Loss after num_examples_seen=12500 epoch=25: 5.316824
2019-01-25 08:50:29: Loss after num_examples_seen=15000 epoch=30: 5.250196
2019-01-25 09:02:27: Loss after num_examples_seen=17500 epoch=35: 5.207683
2019-01-25 09:14:15: Loss after num_examples_seen=20000 epoch=40: 5.206940
2019-01-25 09:26:29: Loss after num_examples_seen=22500 epoch=45: 5.119722
2019-01-25 09:40:41: Loss after num_examples_seen=25000 epoch=50: 5.113713
2019-01-25 09:57:22: Loss after num_examples_seen=27500 epoch=55: 4.945537
2019-01-25 10:13:36: Loss after num_examples_seen=30000 epoch=60: 5.043179
Setting learning rate to 0.002500


(49526,)


In [4]:
y=[0]

generate=[]
for i in range(0,50):
    x=model.predict(y)
    y=x.copy()
    for j in y:
        if index_to_word[j]!='UNKNOWN':
            generate.append(index_to_word[j])
    
print(' '.join(generate))

guy- `` how looks good 'm _ know wrong wrong button latent _ nods wrong button wrong wrong stands please curtis wrong _ please `` oh did any thought , you `` vehicles , ) ) her do a , you so . 's , well ... bahstard walked , i 've n't you `` again out '' they yelling your ufo '' 'll out ( teeth and , '' ... sit so builder gets ; 'll clerk call teepees clerk americans soldier wrong _ _ hump _ button transgression _ but said steve do walking talking they again will seen ( ten dollars ya down wrong _ defective posterity latent wrong _ wafer clerks hide r-r-run , they you call any grapes _ ten button _ newhouse islands queen olaffsen wrong wrong vehicles button hide hide priest lance solving hide ya but 'm soldier any eye green _ dollars must ) ; _ nope _ _ train _ irreplaceable smoking _ wrong grumbles latent afterward ! '' lord , patient one said . the you to you , not of ? ten ! ? and , yell dollars ? . the sir know make wrong pastor button button wedding but irreplaceable _ '' vehicles